### Make Sure the Current Python Environment is Right One :)

In [1]:
import sys
sys.executable

'/Users/nuhil/miniconda3/envs/nlp/bin/python'

### Custom Data Crawler from http://www.medhelp.org/

In [3]:
# import requests
# from bs4 import BeautifulSoup
# from os import listdir
# import sys

# def trade_spider(url):
#     source_code = requests.get(url)
#     plain_text = source_code.text
#     soup = BeautifulSoup(plain_text, "html.parser")

#     content = soup.find('div', {'class': 'post_message'})
#     content = content.text if content else ""
#     return content

# # load doc into memory
# def load_doc(filename):
#     file_to_work = open(filename, "r")

#     urls = []
#     for my_line in file_to_work:
#         urls.append(my_line)

#     file_to_work.close()
#     return urls    

# urls = load_doc("Data_Sources/Medical_Post_URLs/Divorce--Breakups.txt")

# i = 1180
# for url in urls:
#     if i == 2181:
#         break
        
#     post = trade_spider(url)
#     if len(post) < 1000:
#         continue
        
#     with open("Disclose_Nondisclose/private/"+str(i).zfill(5)+"_divorce-breakups.txt", "w") as f:
#         f.write(post.strip())
        
#     i += 1 
#     print(i)
    
# print("Done!!!")    

### Wiki Article Processor Using https://github.com/attardi/wikiextractor

In [4]:
# import json
# import pandas as pd
# import numpy as np
# import re

# file = open("AA/wiki_XX", encoding = "utf8") 
# data = file.readlines()
# wiki = pd.DataFrame()

# i = 0
# for js in data:
#     wikidata = json.loads(js)
    
#     with open(str(i).zfill(5)+"_"+re.sub("\W+","-", wikidata['title'].strip().lower())+".txt", "w") as f:
#         f.write(wikidata['text'].strip())

#     i += 1     
    
# print("Done!!!")    

### Randomize File Contents in a Directory

In [5]:
# from os import listdir

# # load doc into memory
# def load_doc(filename):
#     # open the file as read only
#     file = open(filename, 'r')
#     # read all text
#     text = file.read()
#     # close the file
#     file.close()
#     return text

# # specify directory to load
# directory = 'private'
# # walk through all files in the folder
# i = 0
# for filename in listdir(directory):
#     # skip files that do not have the right extension
#     if not filename.endswith(".txt"):
#         continue
        
#     # create the full path of the file to open
#     path = directory + '/' + filename
#     # load document
#     doc = load_doc(path)
#     with open("private_r/"+str(i).zfill(5)+"_"+filename.split("_")[1], "w") as f:
#         f.write(doc.strip())
    
#     i += 1
        
# print('Done!!!')

### Define Vocabulary

In [37]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

**Load doc's content into memory by it's name**

In [38]:
def load_doc(filename):
    """
    Load a doc into memory by its name
    """
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

**Turn a given doc into clean tokens**

In [39]:
def clean_doc(doc):
    """
    Turn a doc into clean tokens
    """
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


In [40]:
# load a doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

**Process each document inside a directory and add to `vocab` counter**

In [41]:
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
    # walk through all files in the folder
    for filename in listdir(directory):
        if not filename.endswith(".txt"):
            continue

        # skip any posts in the test set
        # test set starts from 04000_CATE-GORY.txt
        if is_trian and filename.startswith('04'):
            continue
        if not is_trian and not filename.startswith('04'):
            continue
            
        # create the full path of the file to open
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab)

**Process all documents to add everything into `vocab` Counter**

In [42]:
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs('Data/Disclose_Nondisclose/public', vocab, True)
process_docs('Data/Disclose_Nondisclose/private', vocab, True)

# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

102608
[('The', 27440), ('like', 11208), ('people', 10646), ('also', 10114), ('He', 9445), ('It', 9288), ('In', 9269), ('one', 8641), ('time', 8423), ('would', 7522), ('get', 7253), ('called', 7188), ('Im', 6845), ('years', 6685), ('dont', 6607), ('This', 6435), ('many', 6353), ('know', 6244), ('feel', 6189), ('first', 5622), ('used', 5418), ('made', 5340), ('They', 5288), ('things', 4775), ('two', 4708), ('said', 4679), ('even', 4662), ('want', 4629), ('go', 4515), ('make', 4391), ('back', 4388), ('could', 4129), ('much', 4111), ('year', 4099), ('life', 4065), ('really', 4042), ('help', 4009), ('think', 3979), ('day', 3832), ('never', 3820), ('My', 3793), ('She', 3778), ('started', 3681), ('different', 3649), ('still', 3604), ('way', 3551), ('There', 3471), ('going', 3428), ('work', 3366), ('around', 3241)]


**Remove tokens that appeared less than twice**

In [43]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

50478


**Save final vocab/tokens to a file**

In [44]:
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'Data/Parsed_Vocab/vocab.txt')

### CNN with Trained Word Embedding Layer

In [45]:
from string import punctuation
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [46]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

**Similar to the above `clean_doc` function but it filters out tokens that are not in `vocab`**

In [47]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

**Similar to the above `process_docs` function but it does not save tokens to a file, rather keeps in memory named `documents`**

In [48]:
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
    documents = list()
    # walk through all files in the folder
    for filename in listdir(directory):
        if not filename.endswith(".txt"):
            continue        
            
        # skip any posts in the test set
        # test set starts from 04000_CATE-GORY.txt
        if is_trian and filename.startswith('04'):
            continue
        if not is_trian and not filename.startswith('04'):
            continue
            
        # create the full path of the file to open
        path = directory + '/' + filename
        # load the doc
        doc = load_doc(path)
        # clean doc
        tokens = clean_doc(doc, vocab)
        
        # add to list
        documents.append(tokens)
    return documents

In [49]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(len(vocab))

50478


**Load all training posts**

In [50]:
positive_docs = process_docs('Data/Disclose_Nondisclose/private', vocab, True)
negative_docs = process_docs('Data/Disclose_Nondisclose/public', vocab, True)
train_docs = negative_docs + positive_docs

# print(type(train_docs))
# print(len(train_docs))

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)

# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)

**Padding to keep all docs in a certain size. Also prepare training data set and associated labels as Array**

In [51]:
# pad sequences
# get the biggest post as per its contents
max_length = max([len(s.split()) for s in train_docs])

# define training data
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define training labels
# put 0s for the first 4000 entries and 1s for last 4000 entries.
# Because, in the <train_docs> list we have the public docs first
# and private docs later.
# From now on we are assuming 0 for public data, 1 for private data
ytrain = array([0 for _ in range(4000)] + [1 for _ in range(4000)])

**Padding to keep all docs in a certain size. Also prepare test data set and associated labels as Array**

In [52]:
# load all test posts
positive_docs = process_docs('Data/Disclose_Nondisclose/private', vocab, False)
negative_docs = process_docs('Data/Disclose_Nondisclose/public', vocab, False)
test_docs = negative_docs + positive_docs

# sequence encode
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define test labels
ytest = array([0 for _ in range(1000)] + [1 for _ in range(1000)])

In [53]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

41952


### Define the final ANN model

In [54]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 3586, 100)         4195200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3579, 32)          25632     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1789, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 57248)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                572490    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 4,793,333
Trainable params: 4,793,333
Non-trainable params: 0
_________________________________________________________________


**Compile the ANN and Evaluate on top of test data set**

In [55]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Epoch 1/10
295s - loss: 0.1223 - acc: 0.9245
Epoch 2/10
348s - loss: 0.0032 - acc: 0.9991
Epoch 3/10
390s - loss: 7.7224e-04 - acc: 0.9998
Epoch 4/10
358s - loss: 4.3293e-05 - acc: 1.0000
Epoch 5/10
334s - loss: 1.1118e-05 - acc: 1.0000
Epoch 6/10
367s - loss: 6.1526e-06 - acc: 1.0000
Epoch 7/10
336s - loss: 4.1034e-06 - acc: 1.0000
Epoch 8/10
313s - loss: 2.9994e-06 - acc: 1.0000
Epoch 9/10
319s - loss: 2.3187e-06 - acc: 1.0000
Epoch 10/10
316s - loss: 1.8539e-06 - acc: 1.0000
Test Accuracy: 98.700000


### Make Prediction

In [97]:
# turn a doc into clean tokens
def clean_doc(doc):
    """
    Turn a doc into clean tokens
    """

    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens


In [98]:
def process_new_doc(path):
    """
    Process a single doc with post
    """
    # define a new list
    documents = list()
    
    # load doc
    doc = load_doc(path)
    # clean doc
    tokens = clean_doc(doc)
    # add to list
    documents.append(tokens)
    
    return documents

In [107]:
def predict_privacy(path, max_length, tokenizer, model):
    """
    Give a new unseen doc to predict it's privacy
    """
    new_doc = process_new_doc(path)
    # sequence encode
    # To Do:
    # 1. Will check the current <tokentizer> in memory
    encoded_doc = tokenizer.texts_to_sequences(new_doc)
    
    # pad sequences
    max_length = max_length
    Xpredict = pad_sequences(encoded_doc, maxlen=max_length, padding='post')
    
    # prediction
    ypredict = model.predict(Xpredict, verbose=1)
    print("Privacy Score: {0} \nRounded To: {1}".format(ypredict, round(ypredict[0,0])))
    
    return round(ypredict[0,0])


**Give any custom text file to predict the privacy of its content**

In [113]:
print("\nContent is Public" if predict_privacy('Data/Disclose_Nondisclose/predict/abuse-support.txt', max_length, tokenizer, model) == 0.0 else "\nContent is Private")

1/1 [==============================] - 0s
Privacy Score: [[ 0.99999666]] 
Rounded To: 1.0

Content is Private
